<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data-Generation" data-toc-modified-id="Data-Generation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Generation</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Performance" data-toc-modified-id="Performance-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Performance</a></span></li><li><span><a href="#Real-Time" data-toc-modified-id="Real-Time-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Real Time</a></span></li><li><span><a href="#&quot;q&quot;-to-stop-webcam" data-toc-modified-id="&quot;q&quot;-to-stop-webcam-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>"q" to stop webcam</a></span></li></ul></div>

# Libraries

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from platform import python_version
print(python_version())

3.7.15


In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh


In [4]:
def extract_keypoints(results):
    """ Extract Landmarks From Face For Traning"""
    for face_landmarks in results.multi_face_landmarks or np.zeros(478*3).flatten():
        faceLandmarks = np.array([[res.x , res.y , res.z ] for res in face_landmarks.landmark]).flatten() if results.multi_face_landmarks else np.zeros(478*3)
        return np.concatenate([faceLandmarks])


In [5]:
#Path for exported data, numpy arrays
DATA_PATH = os.path.join('Head_Face_Mesh_Data')

#Actions that we try to detect 
actions = np.array(['tension','no tension','Head_down'])

#Thirty videos worth of data
no_sequences = 30

# Videos are going to be thirty frames in length
sequence_length = 30

In [6]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Data Generation

In [8]:
label_map = {label:num for num, label in enumerate(actions)}

In [9]:
label_map

{'tension': 0, 'no tension': 1, 'Head_down': 2}

# Data

In [10]:
# DATA_PATH="Head_Face_Mesh_Data_TEST"
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window =[]
        for frame_num in range(no_sequences):
            res = np.load(os.path.join(DATA_PATH,action, str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
            sequences.append(window)
            labels.append(label_map[action])

In [11]:
X = np.asarray(sequences)
y = to_categorical(labels)

In [12]:
X_train , X_test , y_train , y_test = train_test_split(X , y, test_size=0.20)

In [13]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# Modeling

In [14]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1434)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

Wall time: 915 ms


In [15]:
res = [.7,0.2,0.1]
actions[np.argmax(res)]

'tension'

In [16]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [17]:
# model.fit(X_train, y_train, epochs=240, callbacks=[tb_callback])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            383744    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 3)                 9

In [19]:
res = model.predict(X_test)

17/17 [==============================] - 3s 83ms/step


In [20]:
model.save('Testaction.h5')

In [21]:
model.load_weights('Testaction.h5')

In [22]:
yhat = model.predict(X_test)

17/17 [==============================] - 1s 67ms/step


In [23]:
y_true = np.argmax(y_test , axis=1).tolist()
yhat = np.argmax(yhat , axis=1).tolist()

# Performance

In [24]:
accuracy_score(y_true , yhat)

0.3648148148148148

# Real Time

In [25]:
global total_tension_time_MM
global total_neutral_time_MM 
global tension_time_percent
global neutral_time_percent

total_tension_time = []
total_neutral_time = []

In [26]:
global sc ,mn , hr
global Nsc ,Nmn , Nhr
global Hsc ,Hmn , Hhr
sequence = []
sc=0; mn=0; hr=0;
Nsc=0; Nmn=0; Nhr=0;
threshold = 0.2
total_neutral=0
tension_time=0
font = cv2.FONT_HERSHEY_SIMPLEX


# YOU CAN CHANGE WEB CAM YOU OTHER USB CAMERAS FROM THIS STATEMENT
cap = cv2.VideoCapture(0)

global total_tension_time_MM
global total_neutral_time_MM 
global tension_time_percent
global neutral_time_percent


# with mp_face_mesh.FaceMesh()
with mp_face_mesh.FaceMesh(max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:

    while True:
        ret, image = cap.read()

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        results = face_mesh.process(image)

        # Draw the face mesh annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_face_landmarks:
              for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())



        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]
            
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence , axis=0))
            if actions[np.argmax(res)] == 'tension' :
                
#                 global sc ,mn , hr
                sc= sc + 1
                if(sc==60):
                    mn=mn+1
                    sc=0
                if(mn==60):
                    hr=hr+1
                    mn=0
                total_tension_time_MM = hr*60+mn+sc/60
                tension_time = tension_time+total_tension_time_MM
                
                tension_time_percent = total_tension_time_MM*100/60
                
                cv2.putText(image,'%i:%i:%i'%(hr,mn,sc), (3,30),font,1,(0,0,255),2,cv2.LINE_4)
                cv2.putText(image,' '.join(actions[np.argmax(res)]), (10,50),font,1,(0,0,255),2,cv2.LINE_4)
            else:
                if actions[np.argmax(res)] == 'Head_down' :
                    
                    Hsc=0
                    Hmn=0
                    Hhr=0
                    cv2.putText(image,'Show Your face, Your head is down', (3,30),font,1,(0,0,255),2,cv2.LINE_4)
                else:
#                     global Nsc ,Nmn , Nhr
                    Nsc= Nsc + 1
                    if(Nsc==60):
                        Nmn=Nmn+1
                        Nsc=0
                    if(Nmn==60):
                        Nhr=Nhr+1
                        Nmn=0
                    total_neutral_time_MM = Nhr*60+Nmn+Nsc/60
                    total_neutral = total_neutral+total_neutral_time_MM
                    neutral_time_percent = total_neutral_time_MM*100/60
                    
                    cv2.putText(image,'%i:%i:%i'%(Nhr,Nmn,Nsc), (3,30),font,1,(0,0,255),2,cv2.LINE_4)
                    cv2.putText(image,' '.join(actions[np.argmax(res)]), (10,50),font,1,(0,0,255),2,cv2.LINE_4)
                    
        
        
            
        #Show to screen
        cv2.imshow('EMOTION DETECTION', image)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
   
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 39ms/step


1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 38ms/step


In [27]:
print("Total Tension Time in MM:", tension_time)
print("No Tension Time in MM:", total_neutral)

Total Tension Time in MM: 0
No Tension Time in MM: 3146.75


In [1]:
Action_names = ['tension','netural']
# neutral_time_percent=1-tension_time_percent
data_in_percent = [tension_time_percent,neutral_time_percent]
plt.bar(Action_names,data_in_percent)

In [ ]:
tension_time_percent

In [ ]:
neutral_time_percent

# "q" to stop webcam